## Consuming (using) APIs

One side of the API story is utilizing an API to accomplish your task at hand.  Think of an API service as a "database over the web" - you won't always have all of the data you need on hand but that shouldn't stop you from utlizing it.  Alot of types of info can be retrieved over an API.

 - Weather Data
 - Traffic Data
 - Demographic Data
 - Star Wars trivia data
 - Movie Data
 - Sports Data
 - ETC ETC ETC
 
Obviously there is a lot of great information out there that you can access - so lets get started utilizing The Star Wars Api (tm)

The `requests` library is the core of our "API Client"

In [1]:
import requests

To "request" data with the `get` verb just call the `get` method on the requests module with the URL.

In [58]:
response = requests.get("http://swapi.co/api/people/1")

Lets take a look at the raw information that came back from the API.

In [6]:
response.content

b'{"name":"Luke Skywalker","height":"172","mass":"77","hair_color":"blond","skin_color":"fair","eye_color":"blue","birth_year":"19BBY","gender":"male","homeworld":"http://swapi.co/api/planets/1/","films":["http://swapi.co/api/films/7/","http://swapi.co/api/films/6/","http://swapi.co/api/films/3/","http://swapi.co/api/films/2/","http://swapi.co/api/films/1/"],"species":["http://swapi.co/api/species/1/"],"vehicles":["http://swapi.co/api/vehicles/14/","http://swapi.co/api/vehicles/30/"],"starships":["http://swapi.co/api/starships/12/","http://swapi.co/api/starships/22/"],"created":"2014-12-09T13:50:51.644000Z","edited":"2014-12-20T21:17:56.891000Z","url":"http://swapi.co/api/people/1/"}'

Notice that there aren't any HTML tags!  Neat.  This is like asking a website for a piece of data - or asking a website to make a database call for you!

To interpret the data coming over we can use the `json` library.  The `json` library serves many purposes but the 2 most important are to:

 - `loads` -> converts a json string of information into a dict/list (aka native python object)
 - `dumps` -> converts a native python object (dict/list) into a string in order to send it "over the wire"

In [9]:
import json

In [62]:
base_info_bundle = [{"name": "Joel", "id": 42, "eye_color": "green"}]

serialized_info = json.dumps(base_info_bundle)
print(type(serialized_info), serialized_info)

native_info = json.loads(serialized_info)
print(type(native_info), native_info)

<class 'str'> [{"id": 42, "eye_color": "green", "name": "Joel"}]
<class 'list'> [{'id': 42, 'name': 'Joel', 'eye_color': 'green'}]


But thanks to `requests` being an awesome library, maybe we can ask for the json right on the response object.

In [63]:
print(type(response.json()))
print(response.json())

<class 'dict'>
{'hair_color': 'blond', 'vehicles': ['http://swapi.co/api/vehicles/14/', 'http://swapi.co/api/vehicles/30/'], 'starships': ['http://swapi.co/api/starships/12/', 'http://swapi.co/api/starships/22/'], 'height': '172', 'mass': '77', 'created': '2014-12-09T13:50:51.644000Z', 'eye_color': 'blue', 'birth_year': '19BBY', 'gender': 'male', 'skin_color': 'fair', 'edited': '2014-12-20T21:17:56.891000Z', 'homeworld': 'http://swapi.co/api/planets/1/', 'species': ['http://swapi.co/api/species/1/'], 'films': ['http://swapi.co/api/films/7/', 'http://swapi.co/api/films/6/', 'http://swapi.co/api/films/3/', 'http://swapi.co/api/films/2/', 'http://swapi.co/api/films/1/'], 'url': 'http://swapi.co/api/people/1/', 'name': 'Luke Skywalker'}


#### WOAH!

This actually took our `str` of json content and converted it right to a native python object for us - that may be helpful.

Lets start writing a series of functions to make working with the SWAPI easier:

In [19]:
def get_person(person_id):
    return requests.get("http://swapi.co/api/people/{}/".format(person_id)).json()

In [21]:
vader = get_person(4)

In [25]:
vader

{'birth_year': '41.9BBY',
 'created': '2014-12-10T15:18:20.704000Z',
 'edited': '2014-12-20T21:17:50.313000Z',
 'eye_color': 'yellow',
 'films': ['http://swapi.co/api/films/6/',
  'http://swapi.co/api/films/3/',
  'http://swapi.co/api/films/2/',
  'http://swapi.co/api/films/1/'],
 'gender': 'male',
 'hair_color': 'none',
 'height': '202',
 'homeworld': 'http://swapi.co/api/planets/1/',
 'mass': '136',
 'name': 'Darth Vader',
 'skin_color': 'white',
 'species': ['http://swapi.co/api/species/1/'],
 'starships': ['http://swapi.co/api/starships/13/'],
 'url': 'http://swapi.co/api/people/4/',
 'vehicles': []}

In [28]:
def get_movies_for_person(person):
    film_titles = []
    for film in person['films']:
        film_titles.append(requests.get(film).json()['title'])
    return film_titles

In [29]:
get_movies_for_person(vader)

['Revenge of the Sith',
 'Return of the Jedi',
 'The Empire Strikes Back',
 'A New Hope']

In [54]:

def clean_people(people_list):
    return [{"id": int(people['url'].split("/")[-2]), 
            "name": people['name'], 
            "films": people['films']
            } for people in people_list]

def get_all_people():
    return clean_people(requests.get("http://swapi.co/api/people/").json()['results'])


In [55]:
def get_people_details(input_name):
    cleaned_people = get_all_people()
    for people in cleaned_people:
        if people['name'] == input_name:
            return people


In [56]:
get_movies_for_person(get_people_details("Darth Vader"))

['Revenge of the Sith',
 'Return of the Jedi',
 'The Empire Strikes Back',
 'A New Hope']